In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

derrick_rose_tenure = list(range(2009,2023))
start_url = "https://www.basketball-reference.com/players/r/rosede01/gamelog/{}"

In [3]:
for year in derrick_rose_tenure:
    url = start_url.format(year)
    data = requests.get(url)
    with open("1_Season_HTML/Season_{}.html".format(year),"w+",encoding="utf-8") as f:
        f.write(data.text)

for year in derrick_rose_tenure:
    with open("1_Season_HTML/Season_{}.html".format(year),encoding="utf-8") as f:
        page = f.read()
        soup = BeautifulSoup(page,"html.parser")
        season_stats = soup.find(id="pgl_basic")

    season_stats_df = pd.read_html(str(season_stats))[0]
    season_stats_df.to_csv("2_Season_DF/Season_{}.csv".format(year))

In [5]:
for year in derrick_rose_tenure:
    raw_df = pd.read_csv("2_Season_DF/Season_{}.csv".format(year))

    # To remove the unnecessary headers in the table
    raw_df.drop(raw_df.index[raw_df['Rk'] == 'Rk'], inplace=True) 

    # Remove instances of out of order values to make data more general
    def normalise_row(row):
        if row['MP'] == 'Did Not Play':
            return "No"
        if row['MP'] == 'Inactive':
            return "No"
        if row['MP'] == 'Did Not Dress':
            return "No"
        if row['MP'] == 'Not With Team':
            return "No"
        else:
            return "Yes"
    raw_df['Game Played'] = raw_df.apply(lambda row : normalise_row(row), axis=1) 
    
    # Replace with time related Minutes played
    raw_df.loc[raw_df["MP"] == "Did Not Play","MP"]= "00:00" 
    raw_df.loc[raw_df["MP"] == "Inactive","MP"] = "00:00"
    raw_df.loc[raw_df["MP"] == "Did Not Dress","MP"] = "00:00"
    raw_df.loc[raw_df["MP"] == "Not With Team","MP"] = "00:00"
    

    # To split the match results and points in the columnm
    raw_df[['Match Result','Points']] = raw_df['Unnamed: 7'].str.split('(', n=1, expand=True) 
    raw_df["Points"] = raw_df["Points"].str.strip(")")
    
    raw_df[["Minutes Played","Seconds Played"]] = raw_df["MP"].str.split(":",expand=True)

    # Drop all unnecessry columns
    raw_df.drop(['Unnamed: 0','Rk','G','Age','Unnamed: 5','Unnamed: 7','MP'],axis=1,inplace=True) 

    raw_df.to_csv("2_Season_DF/Filtered_Season_{}.csv".format(year))

In [9]:
dtype_dict = {'FG':'int', 'FGA':'int','FG%':'float', '3P':'int', '3PA':'int', '3P%':'float', 'FT':'int', 'FTA':'int', 'FT%':'float',
    'ORB':'int', 'DRB':'int', 'TRB':'int','AST':'int', 'STL':'int', 'BLK':'int', 'TOV':'int', 'PF':'int', 'PTS':'int', 'Minutes Played':'int'}

final_df = pd.DataFrame()
for year in derrick_rose_tenure:
    raw_df = pd.read_csv("2_Season_DF/Filtered_Season_{}.csv".format(year))
    final_df = pd.concat([final_df,raw_df])

final_df.to_csv("3_Final_Dataset/Derrick_Rose_All_Time_Stats.csv")

raw_df = pd.read_csv("3_Final_Dataset/Derrick_Rose_All_Time_Stats.csv")
played_df = raw_df[raw_df['Game Played'] == "Yes"]
played_df.astype(dtype_dict)
played_df.to_csv("3_Final_Dataset/Derrick_Rose_Play_Time_Stats.csv") 

In [5]:
stats_df = pd.read_csv("3_Final_Dataset/Derrick_Rose_Play_Time_Stats.csv")
stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 32 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0.2    672 non-null    int64  
 1   Unnamed: 0.1    672 non-null    int64  
 2   Unnamed: 0      672 non-null    int64  
 3   Date            672 non-null    object 
 4   Tm              672 non-null    object 
 5   Opp             672 non-null    object 
 6   GS              672 non-null    int64  
 7   FG              672 non-null    int64  
 8   FGA             672 non-null    int64  
 9   FG%             671 non-null    float64
 10  3P              672 non-null    int64  
 11  3PA             672 non-null    int64  
 12  3P%             540 non-null    float64
 13  FT              672 non-null    int64  
 14  FTA             672 non-null    int64  
 15  FT%             562 non-null    float64
 16  ORB             672 non-null    int64  
 17  DRB             672 non-null    int

In [7]:
pd.isnull(stats_df).sum()

Unnamed: 0.2        0
Unnamed: 0.1        0
Unnamed: 0          0
Date                0
Tm                  0
Opp                 0
GS                  0
FG                  0
FGA                 0
FG%                 1
3P                  0
3PA                 0
3P%               132
FT                  0
FTA                 0
FT%               110
ORB                 0
DRB                 0
TRB                 0
AST                 0
STL                 0
BLK                 0
TOV                 0
PF                  0
PTS                 0
GmSc                0
+/-                 0
Game Played         0
Match Result        0
Points              0
Minutes Played      0
Seconds Played      0
dtype: int64

In [8]:
stats_df[['Date','TRB', 'AST', 'STL', 'BLK','PTS']]

,Date,TRB,AST,STL,BLK,PTS
0,2008-10-28,4,9,3,0,11
1,2008-10-31,4,1,1,0,18
2,2008-11-01,6,3,0,0,26
3,2008-11-03,4,1,0,0,14
4,2008-11-05,0,7,1,0,20
...,...,...,...,...,...,...
667,2021-12-08,5,7,0,0,8
668,2021-12-10,6,4,0,0,11
669,2021-12-12,2,7,3,0,18
670,2021-12-14,4,6,1,0,15


In [9]:
def FindTrpDoub(lst):
    count = 0
    for value in lst:
        if value >=10:
            count+=1
    if count >= 3:
        return "Yes"
    else:
        return "No"  
    
trp_doub_apply = stats_df.apply(lambda row:FindTrpDoub(row[['TRB', 'AST', 'STL', 'BLK','PTS']]),axis=1)
stats_df["Trp Dbl"] = trp_doub_apply

In [10]:
stats_df.loc[stats_df["Trp Dbl"] == "Yes"]


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Date,Tm,Opp,GS,FG,FGA,FG%,...,PF,PTS,GmSc,+/-,Game Played,Match Result,Points,Minutes Played,Seconds Played,Trp Dbl
198,204,40,42,2011-01-17,CHI,MEM,1,7,20,0.35,...,2,22,22.1,13,Yes,W,12,39,45,Yes


In [33]:
def FillTeamName(value):
    full_name_dict = {'CHI':"Illinois","CLE":"Ohio","DET":"Michigan","MIN":"Minnesota","NYK":"New York"}
    for key_val in full_name_dict.keys():
        if key_val == value:
            return full_name_dict[value]
    
full_team_name = stats_df.apply(lambda row:FillTeamName(row['Tm']),axis=1)
stats_df["Team"] = full_team_name
stats_df["Team"].describe()

count          672
unique           5
top       Illinois
freq           406
Name: Team, dtype: object

In [35]:
stats_df.to_csv("3_Final_Dataset/Derrick_Rose_Play_Time_Stats.csv")
